In [1]:
import data
import torch
import torch.utils.data
import numpy as np
import gans.w_gan as wgan
import gans.w_gp_gan as wgpgan
import gans.vae as vae
import gans.ns_gan as nsgan
import gans.mm_gan as mmgan
import gans.ls_gan as lsgan
import gans.dra_gan as dragan
import gans.be_gan as began
from gans.load_data import get_data

In [2]:
i = 0
j = 0

# Multivariate Distributions

In [3]:
distributions = ['normal', 'beta', 'exponential', 'gamma', 'gumbel', 'laplace']
generator = data.Distribution(distributions[i], 200)
data_train = generator.generate_samples(50000)
data_test = generator.generate_samples(20000)
# generator.save_dist('output_file.pickle')  # Saving the generator

# Mixture Models

In [4]:
distributions = ['normal', 'beta', 'exponential', 'gamma', 'gumbel', 'laplace']
pdfs = ['uniform', 'random']
generator = data.MixtureDistribution(distributions[i], distributions[j], n_mixtures=15, dim=200)
data_train = generator.generate_samples(50000)
data_test = generator.generate_samples(20000)
# generator.get_log_likelihood(data_train)
# generator.save_dist()

# Circles

In [5]:
generator = data.CirclesDatasetGenerator(size=256, n_circles=4, random_colors=True, random_sizes=True, modes=20)
x = generator.generate_samples(10000)
print(len(x))
print(x[0].shape)
# generator.generate_samples_to_directory(5000, './dataset_1')
# generator.save_generator('./generator1.pickle')

10000
(256, 256, 3)


# Piping to the GANs

In [6]:
i = 0
distributions = ['normal', 'beta', 'exponential', 'gamma', 'gumbel', 'laplace']
generator = data.Distribution(distributions[i], 200)
data_train = generator.generate_samples(50000)
data_test = generator.generate_samples(20000)

In [7]:
x_torch = torch.from_numpy(data_train)

In [8]:
dataset = torch.utils.data.TensorDataset(x_torch)

TypeError: __init__() missing 1 required positional argument: 'target_tensor'

In [9]:
# Test on MNIST
train_iter, val_iter, test_iter = get_data()

# WGAN

In [ ]:
model = wgan.WGAN(image_size = 784, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = wgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 2, G_lr = 5e-5, D_lr = 5e-5, D_steps = 5, clip = 0.01)

# WGAN GP

In [ ]:
model = wgpgan.WGANGP(image_size = 784, hidden_dim = 256, z_dim = 128) 
if torch.cuda.is_available():
    model = model.cuda()
trainer = wgpgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 25, G_lr = 1e-4, D_lr = 1e-4, D_steps = 5)

# VAE

In [ ]:
model = vae.VAE(image_size = 784, hidden_dim = 400, z_dim = 20)
if torch.cuda.is_available():
    model.cuda()
trainer = vae.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model, 40)

# Explore latent space
viz = Viz(model)
viz.make_all()

# NS GAN

In [ ]:
model = nsgan.NSGAN(image_size = 784, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = nsgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 25, G_lr = 2e-4, D_lr = 2e-4, D_steps = 1)

# MM GAN

In [ ]:
model = mmgan.MMGAN(image_size = 784, hidden_dim = 400, z_dim = 100)
if torch.cuda.is_available():
    model = model.cuda()
trainer = mmgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 25, G_lr = 2e-4, D_lr = 2e-4, G_init = 5, D_steps = 1)

# LS GAN

In [ ]:
model = lsgan.LSGAN(image_size = 784, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = lsgan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 25, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)

# DRA GAN

In [ ]:
model = dragan.DRAGAN(image_size = 784, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = dragan.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 25, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)

# BE GAN

In [ ]:
model = began.BEGAN(image_size = 784, hidden_dim = 256, z_dim = 128)
if torch.cuda.is_available():
    model = model.cuda()
trainer = began.Trainer(train_iter, val_iter, test_iter)
model = trainer.train(model = model, num_epochs = 100, G_lr = 1e-4, D_lr = 1e-4, D_steps = 1)